## Multi-mode data

Finally we repeat the modelling part of this notebook using in this case multiple modes (fundamental, 1st, and 2nd).

In [2]:
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pylops
import json
from pathlib import Path
from PIL import Image
import torch
import torch.nn.functional as F

from functools import partial
from scipy.optimize import minimize, Bounds
from disba import PhaseDispersion

from surfacewaves import *
from dispersionspectra import *
from inversion import *
from utils import gen_model1

import ccfj
import scipy
from Dispersion.dispersion import get_dispersion


def get_cpr(thick, vs, periods, n_mode=3):

    thick_true,vp_true,vs_true,rho_true = gen_model1(thick,vs,area=True)
    true_model = np.vstack([thick_true, vp_true, vs_true, rho_true]).T
    # Rayleigh-wave fundamental model dispersion curve 
    pd = PhaseDispersion(*true_model.T)
    cpr = [pd(periods[imode], mode=imode, wave="rayleigh") for imode in range(n_mode)]

    return cpr

def random_thick_vs(thick, vs, periods, fluctuation_percentage=0.1, n_mode=3):
    # 生成浮动值
    random_thick = thick * (1 + fluctuation_percentage * (2 * np.random.rand(len(thick)) - 1))
    random_vs = vs * (1 + fluctuation_percentage * (2 * np.random.rand(len(vs)) - 1))

    try:
        cpr = get_cpr(random_thick, random_vs, periods, n_mode)
        return cpr, random_thick, random_vs
    except Exception as e:
        print(e)

def get_ccf(nt, dt, nx, dx, nfft, cpr, n_mode=3):
    t, x = np.arange(nt)*dt, np.arange(nx)*dx
    # Wavelet
    wav = ormsby(t[:nt//2+1], f=[2, 4, 38, 40], taper=np.hanning)[0][:-1]
    wav = np.roll(np.fft.ifftshift(wav), 20) # apply small shift to make it causal

    # Data
    dshifts, fs, vfs = [], [], []
    for imode in range(n_mode):
        dshift_, f_, vf_ = surfacewavedata(nt, dt, nx, dx, nfft, 
                                        np.flipud(1/cpr[imode][0]), np.flipud(cpr[imode][1]), wav)
        dshifts.append(1./(imode+1)**0.1 * dshift_[np.newaxis])
        fs.append(f_)
        vfs.append(vf_)
    dshift = np.concatenate(dshifts).sum(0)
    return dshift, fs, vfs

def get_disp(nt, dt, nx, dx, nfft, cpr, fmins, fmaxs, 
            fmin, fmax, cmin, cmax, dc, n_mode=3):
    t, x = np.arange(nt)*dt, np.arange(nx)*dx
    # Wavelet
    wav = ormsby(t[:nt//2+1], f=[2, 4, 38, 40], taper=np.hanning)[0][:-1]
    wav = np.roll(np.fft.ifftshift(wav), 20) # apply small shift to make it causal

    outs, fs, vfs = [], [], []
    for imode in range(n_mode):
        dshift_, f_, vf_ = surfacewavedata(nt, dt, nx, dx, nfft, 
                                        np.flipud(1/cpr[imode][0]), np.flipud(cpr[imode][1]), wav)
        # dshifts.append(1./(imode+1)**0.1 * dshift_[np.newaxis])
        f, c, out, U, t = get_dispersion(dshift_.T, dx, dt, 
                                            cmin, cmax, dc, fmin, fmax)
        ind_fmax = np.argmin(np.abs(f-fmaxs[imode]))+1
        ind_fmin = np.argmin(np.abs(f-fmins[imode]))
        out[:, :ind_fmin] = 0
        out[:, ind_fmax:] = 0
        outs.append(out)
        fs.append(f_)
        vfs.append(vf_)
    outs = np.mean(outs, axis=0)

    return outs, fs, vfs

def park(dshift, dx, dt, cmin, cmax, dc, fmin, fmax):
    f1, c1, img, U, t = get_dispersion(dshift.T, dx, dt, 
                                        cmin, cmax, dc, fmin, fmax)

    return f1, c1, img, U, t

def fj(dshift, dx, dt, cmin, cmax):
    nx, nt = dshift.shape
    x = np.arange(nx)*dx
    f = scipy.fftpack.fftfreq(nt,dt)[:nt//2]
    c = np.linspace(cmin, cmax, 1000)

    out = ccfj.fj_earthquake(dshift,x,c,f,fstride=1,itype=0,func=0)
    
    return f, c, out

def show_fj(f, c, out, fmin, fmax, ii, aa):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(out, aspect='auto', cmap='gray',
            extent=(f.min(), f.max(),c.min(), c.max()),origin='lower')

    ax.margins(0)
    ax.set_xlim(fmin, fmax)
    ax.set_ylim(c.min(), c.max())
    fig.savefig(f'/home/lty/MyProjects/Seismology/diffseis/dataset/demultiple/data_train/data/{aa}{ii:03d}.png', 
                dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()

def show_label(f, c, out, cpr, fmin, fmax, ii, aa):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(np.zeros_like(out), aspect='auto', cmap='gray',
            extent=(f.min(), f.max(),c.min(), c.max()),origin='lower')
    for imode in range(3):
        ax.plot(np.flipud(1/cpr[imode][0]), 1.e3*np.flipud(cpr[imode][1]), 
                    'white', lw=4)

    ax.margins(0)
    ax.set_xlim(fmin, fmax)
    ax.set_ylim(c.min(), c.max())

    fig.savefig(f'/home/lty/MyProjects/Seismology/diffseis/dataset/demultiple/data_train/labels/{aa}{ii:03d}.png', 
                dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()


def save_image(image_numpy, image_path, aspect_ratio=1.0):
    """Save a numpy image to the disk

    Parameters:
        image_numpy (numpy array) -- input numpy array
        image_path (str)          -- the path of the image
    """

    image_pil = Image.fromarray(image_numpy)
    h, w, _ = image_numpy.shape

    if aspect_ratio is None:
        pass
    elif aspect_ratio > 1.0:
        image_pil = image_pil.resize((h, int(w * aspect_ratio)), Image.BICUBIC)
    elif aspect_ratio < 1.0:
        image_pil = image_pil.resize((int(h / aspect_ratio), w), Image.BICUBIC)
    image_pil.save(image_path)

# 假设 record 中包含 ndarray，你可以先进行转换
def convert_ndarray(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_ndarray(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_ndarray(i) for i in obj]
    else:
        return obj


def show_out(out, name):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(out, aspect='auto', origin='lower', cmap='gray')
    fig.savefig(name, dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()

def show_mask(mask, name):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(mask, aspect='auto', origin='lower', cmap='gray')
    fig.savefig(name, dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()

def show_line(f, c, out, fff, ccc, name):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.axis("off")
    ax.imshow(np.zeros_like(out), aspect='auto', cmap='gray',
            extent=(fff.min(), fff.max(),ccc.min(), ccc.max()),origin='lower')
    ax.plot(f, c, 'white', lw=2)

    fig.savefig(name, dpi=300,bbox_inches='tight', pad_inches=0)
    plt.close()

/csim2/zhangzhiyu/miniconda3/envs/ccfj/lib/python3.12/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [3]:
import numpy as np
import random
import concurrent.futures


# Frequency axis
# fdisp = np.linspace(3, 40, 81)
# period = np.flipud(1/fdisp)         # Periods (must be sorted starting with low periods)

f0_min, f0_max = 3, 20
f1_min, f1_max = 10, 25
f2_min, f2_max = 15, 30
f3_min, f3_max = 20, 35
f4_min, f4_max = 25, 40
fmins = [f0_min, f1_min, f2_min, f3_min, f4_min]
fmaxs = [f0_max, f1_max, f2_max, f3_max, f4_max]

fdisp1 = np.linspace(f0_min, f0_max, 3*(f0_max-f0_min))
fdisp2 = np.linspace(f1_min, f1_max, 3*(f1_max-f1_min))
fdisp3 = np.linspace(f2_min, f2_max, 3*(f2_max-f2_min))
fdisp4 = np.linspace(f3_min, f3_max, 3*(f3_max-f3_min)) 
fdisp5 = np.linspace(f4_min, f4_max, 3*(f4_max-f4_min)) 
fdisps = [fdisp1, fdisp2, fdisp3, fdisp4, fdisp5]
periods = [np.flipud(1/fdisp1), np.flipud(1/fdisp2), np.flipud(1/fdisp3), np.flipud(1/fdisp4), np.flipud(1/fdisp5)]

# Axes
nt = 600 # number of time samples
dt = 0.008 # time sampling in s
nx = 81 # number of spatial samples
dx = 2.5 # spatial sampling in m
nfft = 2**10

dc = 3.
cmin, cmax = 50., 1000.
# fmin, fmax = fdisp.min(), fdisp.max()
fmin = np.min([np.min(fdisp) for fdisp in fdisps])
fmax = np.max([np.max(fdisp) for fdisp in fdisps])



# 原始数组
# thick = np.array([0.01, 0.02, 0.03, 0.01])
# vs = np.array([0.2, 0.4, 0.6, 0.8])
fluctuation_percentage = 0.2        # 定义浮动范围百分比


In [4]:
for _ in range(50):
    h1, h2, h3, h4 = 0.02, 0.02, 0.03, 1.0
    v1, v2, v3, v4 = 0.3, 0.7, 0.5, 1.6
    n_mode = 1

    thick = np.array([h1, h2, h3, h4])
    vs = np.array([v1, v2, v3, v4])
    for ii in range(2):
        cpr, random_thick, random_vs = random_thick_vs(thick, vs, periods, 
                                                       fluctuation_percentage, n_mode=n_mode)
        # cmin = np.min([np.min(cpr[imode][1]) for imode in range(3)]) * 1e3
        cmax = np.max([np.max(cpr[imode][1]) for imode in range(n_mode)]) * 1e3 + 100
        out, fs, vfs = get_disp(nt, dt, nx, dx, nfft, cpr, fmins, fmaxs, 
                                 fmin, fmax, cmin, cmax, dc, n_mode=n_mode)

        aa = f"{''.join(f'{v:.5f}_{h:.5f}_' for v, h in zip(random_vs, random_thick))}"
        path = Path('/csim2/zhangzhiyu/MyProjects/DiffDispHighMode/Dispersion/dataset_png/train/')
        name = path / f'data/{aa}.png'
        if name.exists():
            print(ii)
            continue

        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        ax.axis("off")
        ax.imshow(out, aspect='auto', origin='lower', cmap='gray', 
                  extent=[fmin, fmax, cmax, cmin])
        fig.savefig(name, dpi=300,bbox_inches='tight', pad_inches=0)
        plt.close()

        # 获得频散曲线坐标
        ff = [np.flipud(1/cpr[imode][0]) for imode in range(n_mode)]
        cc = [1.e3*np.flipud(cpr[imode][1]) for imode in range(n_mode)]
        for ind, (f1, c1) in enumerate(zip(ff, cc)):
            name = name = path / f'labels/{aa}_{ind+1:02d}.png'
            fig = plt.figure()
            ax = fig.add_axes([0,0,1,1])
            ax.imshow(np.zeros_like(out), aspect='auto', cmap='gray', 
                  extent=[fmin, fmax, cmin, cmax])
            ax.axis("off")
            ax.plot(f1, c1, color='white', lw=2.5)
            # ax.set_xlim(fmin, fmax)
            # ax.set_ylim(cmin, cmax)
            fig.savefig(name, dpi=300,bbox_inches='tight', pad_inches=0)
            plt.close()


In [13]:
from pathlib import Path
data_dir = "/csim2/zhangzhiyu/MyProjects/DiffDispHighMode/Dispersion/dataset_png/train"

files = list(Path(data_dir, 'data').glob('*.png'))
len(files)

1506

In [146]:
# from pathlib import Path
# import re

# data_dir = "/csim2/zhangzhiyu/MyProjects/DiffDispHighMode/Dispersion/dataset_png/train"
# data_path = Path(data_dir, 'labels')

# files = list(data_path.glob('*.png'))

# pattern = re.compile(r'_(\d{2})\.png$')

# for file in files:
#     match = pattern.search(file.name)
#     if match:
#         ind = int(match.group(1))
#         new_ind = ind + 1
#         new_name = file.name.replace(f'_{ind:02d}.png', f'_{new_ind:02d}.png')
#         new_file = file.with_name(new_name)
#         file.rename(new_file)


In [1]:
# import shutil

# import os

# # 删除 data 和 labels 文件夹下的所有图片文件（.png）
# data_dir = '/csim2/zhangzhiyu/MyProjects/DiffDispHighMode/Dispersion/dataset_png/train/data'
# labels_dir = '/csim2/zhangzhiyu/MyProjects/DiffDispHighMode/Dispersion/dataset_png/train/labels'

# def remove_png_files(directory):
#     if os.path.exists(directory):
#         for filename in os.listdir(directory):
#             if filename.endswith('.png'):
#                 file_path = os.path.join(directory, filename)
#                 try:
#                     os.remove(file_path)
#                 except Exception as e:
#                     print(f"无法删除 {file_path}: {e}")

# remove_png_files(data_dir)
# remove_png_files(labels_dir)

